In [1]:
import pickle
import pandas as pd
import mysql.connector
from scipy.spatial import distance
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def read_pkl_file(file_name):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    return data

In [3]:
def SceneGraphDataset_to_df(data):
    graph_dict = dict()
    scene_key = data.scene_graphs.keys()
    for x in scene_key:
        for key in data.scene_graphs[x].keys():
            try:
                graph = data.scene_graphs[x][key].g
                edges = list(graph.edges(data=True, keys=True))
                nodes = list(graph.nodes(data=True))
                graph_dict[(x,key)] = (nodes,edges)
            except Exception as e:
                print("missing nodes or edges:", str(e))
    df = pd.DataFrame(graph_dict).transpose()
    df.columns = ['Nodes', 'Edges']
    return df     

In [4]:
def node_edge_extraction(pd_series):
    nodes,edges = [],[]
    for node in pd_series.Nodes:
        nodes.append(node[0])
    for edge in pd_series.Edges:
        edges.append((edge[0],edge[1],edge[3]['label']))
    return nodes,edges
    

In [5]:
def drop_all_tables(d,cursor):
    try:
        for name in d.keys():
            # Construct the table name
            table_name = f"table_{name}"
            
            # Create the table
            create_table_query = f"""
            Drop table {table_name}
            """
            cursor.execute(create_table_query)
    except:
        print('error')


In [6]:
#load pickle file to dataframe
file_name = '271_carla_scenegraphs.pkl'
data = read_pkl_file(file_name)
df = SceneGraphDataset_to_df(data)

In [9]:
#process clean the dataframe
df = df.reset_index()
df['id'] = df.apply(lambda row: (row['level_0'], row['level_1']), axis=1)
df = df.drop(columns =['level_0','level_1'])

In [10]:
nodes_l = df['Nodes']
new_nodes_l = []
for i in range(len(nodes_l)):
    l=[]
    for x in nodes_l[i]:
        l.append((x[0].name,x[1]['attr']))
    new_nodes_l.append(l)
df['Nodes']= new_nodes_l

In [ ]:
df['id']=df['id'].astype('str')
df[df['id'].str.startswith('(0, 1779')]['Nodes'][46]

In [ ]:
df[df['id'].str.startswith('(0, 1779')]['Edges'][46]

In [37]:
#expand nodes
df_nodes= df.explode('Nodes').reset_index(drop=True)
df_nodes =df_nodes[['Nodes','id']]

In [38]:
extend_df = pd.DataFrame(df_nodes['Nodes'].apply(lambda x: pd.Series((x[0], *x[1].values()))))

In [ ]:
extend_df.columns = ['Name', 'velocity_abs','velocity','location','rotation','ang_velocity','name','lane_id','road_id','left_blinker_on','right_blinker_on','brake_light_on','lane_idx','orig_lane_idx','invading_lane']
extend_df['id'] = df_nodes['id']
extend_df = extend_df[~extend_df['Name'].isin(['Root Road', 'lane_left','lane_right','lane_middle'])]

df_nodes = extend_df
df_nodes['isEgo'] = 0
df_nodes.loc[df_nodes['Name'].str.contains('ego'), 'isEgo'] = 1
df_nodes[['Name','isEgo']]
df_nodes['isEgo'] = df_nodes['isEgo'].map({0: False, 1: True})
df_nodes = df_nodes.fillna(0)

In [85]:
ego_cars = df_nodes[df_nodes['isEgo']]
ego_cars = ego_cars[['id','Name','velocity_abs','velocity','location','rotation']]

In [86]:
ndf= df_nodes.merge(ego_cars,on=['id'],suffixes=('', '_ego'))

In [87]:
def calculate_distance(point1, point2):
    return distance.euclidean(point1, point2)

ndf['abs_distance'] = ndf.apply(lambda row: calculate_distance(row['location'], row['location_ego']), axis=1)

In [89]:
ndf['velocity_diff']= ndf['velocity_abs'] -  ndf['velocity_abs_ego']

In [91]:
ndf.to_csv('df_nodes.csv', index=False)

In [28]:
n_node_df =ndf[['Name','name','velocity_abs','abs_velocity_diff','location','abs_distance','lane_id','road_id','id','isEgo','rotation']]
n_node_df['car_id']  = n_node_df.apply(lambda row: str((row['Name'], row['id'])), axis=1)

/var/folders/n_/8gt809p93q5bs3ts1z8by0s00000gn/T/ipykernel_1945/3920807546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_node_df['car_id']  = n_node_df.apply(lambda row: str((row['Name'], row['id'])), axis=1)


In [24]:
df_edges =  df[['Edges','id']]
df_edges= df_edges.explode('Edges').reset_index(drop=True)
df_edges[['Node_1', 'Node_2', 'value', 'label']] = df_edges['Edges'].apply(lambda x: pd.Series((x[0].name, x[1].name, x[3]['value'], x[3]['label'])))
df_edges.drop(['Edges'], axis=1, inplace=True)
#filter only cars
'''df_edges = df_edges[~(df_edges['Node_1'].str.contains('lane') | df_edges['Node_2'].str.contains('lane'))]
df_edges = df_edges[~(df_edges['Node_1'].str.contains('Root') | df_edges['Node_2'].str.contains('Root'))]
'''

"df_edges = df_edges[~(df_edges['Node_1'].str.contains('lane') | df_edges['Node_2'].str.contains('lane'))]\ndf_edges = df_edges[~(df_edges['Node_1'].str.contains('Root') | df_edges['Node_2'].str.contains('Root'))]\n"

In [29]:
df_edges['label'].unique()

array(['isIn', 'near_coll', 'atDRearOf', 'toLeftOf', 'super_near',
       'inDFrontOf', 'toRightOf', 'very_near', 'atSRearOf', 'inSFrontOf',
       'near', 'visible'], dtype=object)

In [33]:
df_edges[df_edges['label']=='isIn'].to_csv('isIn.csv')

In [18]:
df_edges['near_level'] = 0 
df_edges.loc[df_edges['label'] == 'near_coll', 'near_level'] = 5
df_edges.loc[df_edges['label'] == 'super_near', 'near_level'] = 4
df_edges.loc[df_edges['label'] == 'very_near', 'near_level'] = 3
df_edges.loc[df_edges['label'] == 'near', 'near_level'] = 2
df_edges.loc[df_edges['label'] == 'visible', 'near_level'] = 1

In [21]:
procees_df = df_edges
procees_df=procees_df.drop('value',axis=1)
grouped = procees_df.groupby('label')
table_df_dict = dict()
for group_name, group_df in grouped:
    table_df_dict[group_name] = group_df
    

In [22]:
#dictionary stores the value of tables
table_df_dict.keys()

dict_keys(['atDRearOf', 'atSRearOf', 'inDFrontOf', 'inSFrontOf', 'isIn', 'near', 'near_coll', 'super_near', 'toLeftOf', 'toRightOf', 'very_near', 'visible'])

In [34]:
n_node_df['Index'] = n_node_df.reset_index().index
n_node_df[['scene_id', 'frame_id']] = pd.DataFrame(n_node_df['id'].tolist(), index=n_node_df.index)
ego_df =  n_node_df[n_node_df['Name']=='ego car']
ego_df['Index'] = ego_df.reset_index().index
car_df = n_node_df[n_node_df['Name']!='ego car']
car_df['Index'] = car_df.reset_index().index

/var/folders/n_/8gt809p93q5bs3ts1z8by0s00000gn/T/ipykernel_1945/3703168377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_node_df['Index'] = n_node_df.reset_index().index
/var/folders/n_/8gt809p93q5bs3ts1z8by0s00000gn/T/ipykernel_1945/3703168377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_node_df[['scene_id', 'frame_id']] = pd.DataFrame(n_node_df['id'].tolist(), index=n_node_df.index)
/var/folders/n_/8gt809p93q5bs3ts1z8by0s00000gn/T/ipykernel_1945/3703168377.py:2: SettingWithCopyWarning: 
A

In [ ]:
#create connection to database
database_name = 'test'
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database=database_name
)
cursor = connection.cursor()

In [470]:
#table for cars
create_table_query = """
CREATE TABLE cars (
    car_id INT NOT NULL,
    velocity_abs FLOAT,
    name VARCHAR(100),
    lane_id FLOAT,
    road_id FLOAT,
    PRIMARY KEY (car_id)
);
"""
cursor.execute(create_table_query)

In [471]:
#table for ego cars
create_table_query = """
CREATE TABLE ego_cars (
    car_id INT NOT NULL,
    velocity_abs FLOAT,
    name VARCHAR(100),
    lane_id FLOAT,
    road_id FLOAT,
    PRIMARY KEY (car_id)
);
"""
cursor.execute(create_table_query)

In [479]:
for name in t_table_df_dict.keys():
    if 'near' not in name:
        # Construct the table name
        table_name = f"table_{name}"
        # Create the table
        create_table_query = f"""
        CREATE TABLE {table_name} (
            ego_car_id INT,
            car_id INT,
            PRIMARY KEY (ego_car_id, car_id),
            FOREIGN KEY(ego_car_id)  REFERENCES ego_cars(car_id),
            FOREIGN KEY(car_id)  REFERENCES cars(car_id)
        )
        """
        cursor.execute(create_table_query)
        print(f"Table {table_name} created successfully.")

connection.commit()

Table table_atDRearOf created successfully.
Table table_atSRearOf created successfully.
Table table_inDFrontOf created successfully.
Table table_inSFrontOf created successfully.
Table table_isIn created successfully.
Table table_toLeftOf created successfully.
Table table_toRightOf created successfully.
Table table_visible created successfully.


In [480]:
create_table_query = f"""
CREATE TABLE near (
    ego_car_id INT,
    car_id INT,
    abs_distance float,
    near_level INT,
    PRIMARY KEY (ego_car_id, car_id),
    FOREIGN KEY(ego_car_id)  REFERENCES ego_cars(car_id),
    FOREIGN KEY(car_id)  REFERENCES cars(car_id)
)
"""
cursor.execute(create_table_query)
print(f"Table {table_name} created successfully.")

Table table_visible created successfully.


In [362]:
'''l = car_df['id'].unique()[0:10]
t_car_df = car_df[car_df['id'].isin(l)]
t_ego_car_df = ego_df[ego_df['id'].isin(l)]

t_df_edge = df_edges[df_edges['id'].isin(l)]
t_df_edge =t_df_edge[t_df_edge['Node_1']=='ego car']
merged_df = pd.merge(t_df_edge, df_nodes[['Name','id','lane_id']], left_on=['id','Node_1'], right_on= ['id','Name'],suffixes=('', '_Node_1'))
merged_df = pd.merge(merged_df, df_nodes[['Name','id','lane_id']], left_on=['id','Node_2'], right_on= ['id','Name'],suffixes=('', '_Node_2'))
merged_df['lane_id_diff'] = merged_df.apply(lambda row: row['lane_id'] -row['lane_id_Node_2'], axis=1)
t_df_edge = merged_df[['id','Node_1','Node_2','label','lane_id_diff']]
t_df_edge =pd.merge(t_df_edge,t_car_df[['id','Name','abs_velocity_diff','abs_distance']], left_on=['id','Node_2'],right_on=['id','Name'])
t_df_edge['near_level'] = 0 
t_df_edge.loc[t_df_edge['label'] == 'near_coll', 'near_level'] = 5
t_df_edge.loc[t_df_edge['label'] == 'super_near', 'near_level'] = 4
t_df_edge.loc[t_df_edge['label'] == 'very_near', 'near_level'] = 3
t_df_edge.loc[t_df_edge['label'] == 'near', 'near_level'] = 2
t_df_edge.loc[t_df_edge['label'] == 'visible', 'near_level'] = 1
'''

In [476]:
l = car_df['id'].unique()[0:10]
'''t_car_df = car_df[car_df['id'].isin(l)]
t_ego_car_df = ego_df[ego_df['id'].isin(l)]
t_df_edge = df_edges[df_edges['id'].isin(l)]'''

t_car_df= car_df
t_ego_car_df = ego_df
t_df_edge = df_edges

t_df_edge =t_df_edge[t_df_edge['Node_1']=='ego car']
merged_df = pd.merge(t_df_edge, df_nodes[['Name','id','lane_id']], left_on=['id','Node_1'], right_on= ['id','Name'],suffixes=('', '_Node_1'))
merged_df = pd.merge(merged_df, df_nodes[['Name','id','lane_id']], left_on=['id','Node_2'], right_on= ['id','Name'],suffixes=('', '_Node_2'))
merged_df['lane_id_diff'] = merged_df.apply(lambda row: row['lane_id'] -row['lane_id_Node_2'], axis=1)
t_df_edge = merged_df[['id','Node_1','Node_2','label','lane_id_diff']]
t_df_edge =pd.merge(t_df_edge,t_car_df[['id','Name','abs_velocity_diff','abs_distance']], left_on=['id','Node_2'],right_on=['id','Name'])
t_df_edge['near_level'] = 0 
t_df_edge.loc[t_df_edge['label'] == 'near_coll', 'near_level'] = 5
t_df_edge.loc[t_df_edge['label'] == 'super_near', 'near_level'] = 4
t_df_edge.loc[t_df_edge['label'] == 'very_near', 'near_level'] = 3
t_df_edge.loc[t_df_edge['label'] == 'near', 'near_level'] = 2
t_df_edge.loc[t_df_edge['label'] == 'visible', 'near_level'] = 1


In [ ]:
procees_df = t_df_edge
grouped = procees_df.groupby('label')
t_table_df_dict = dict()
for group_name, group_df in grouped:
    t_table_df_dict[group_name] = group_df
    

In [478]:
t_table_df_dict.keys()

dict_keys(['atDRearOf', 'atSRearOf', 'inDFrontOf', 'inSFrontOf', 'isIn', 'near', 'near_coll', 'super_near', 'toLeftOf', 'toRightOf', 'very_near', 'visible'])

In [481]:
#load data into the cars table
for _, row in t_car_df.iterrows():
    query = "INSERT INTO cars (car_id, velocity_abs,name,lane_id,road_id) VALUES (%s,%s,%s,%s,%s)"
    values = (row['Index'],row['velocity_abs'],row['Name'],row['lane_id'],row['road_id'])
    cursor.execute(query, values)

connection.commit()

In [482]:
#load data into the cars table
for _, row in t_ego_car_df.iterrows():
    query = "INSERT INTO ego_cars (car_id, velocity_abs,name,lane_id,road_id) VALUES (%s,%s,%s,%s,%s)"
    values = (row['Index'],row['velocity_abs'],row['Name'],row['lane_id'],row['road_id'])
    cursor.execute(query, values)

connection.commit()

In [36]:
t_table_df_dict.keys()

NameError: name 't_table_df_dict' is not defined

In [ ]:
for name in t_table_df_dict.keys():
    #insert the value
    if 'near' not in name and 'visible' not in name:
        c_df = t_table_df_dict[name]
        
        merged_df = pd.merge(c_df, t_car_df[['id', 'Name','Index']], left_on=['id', 'Node_2'],right_on=['id', 'Name'],suffixes=('','_car'))
        merged_df = pd.merge(merged_df, t_ego_car_df[['id', 'Name','Index']], left_on=['id', 'Node_1'],right_on=['id', 'Name'],suffixes=('','_ego'))
        for index, row in merged_df.iterrows():
            try:
                table_name = "table_" + row['label']
                insert_query = "INSERT INTO " + table_name + " (ego_car_id, car_id " + ") VALUES (%s, %s)"
                values = (str(row['Index_ego']), str(row['Index']))
                cursor.execute(insert_query, values)
            except:
                print('error')
    
connection.commit()

In [491]:
for name in t_table_df_dict.keys():
    #insert the value
    if 'near' in name or 'visible' in name:
        c_df = t_table_df_dict[name]
        merged_df = pd.merge(c_df, t_car_df[['id', 'Name','Index']], left_on=['id', 'Node_2'],right_on=['id', 'Name'],suffixes=('','_car'))
        merged_df = pd.merge(merged_df, t_ego_car_df[['id', 'Name','Index']], left_on=['id', 'Node_1'],right_on=['id', 'Name'],suffixes=('','_ego'))
        table_name = "near"
        for index, row in merged_df.iterrows():
            try:
                insert_query = "INSERT INTO " + table_name + " (ego_car_id, car_id, " + "abs_distance" + ", " + "near_level" + ") VALUES (%s, %s, %s, %s)"
                values = (str(row['Index_ego']), str(row['Index']), row['abs_velocity_diff'], row['near_level'])
                cursor.execute(insert_query, values)
            except:
                print('error')

connection.commit()